<img src="images/valueiteration.png" width="500">

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Create-Environment" data-toc-modified-id="Create-Environment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Create Environment</a></span></li><li><span><a href="#Value-iteration-function" data-toc-modified-id="Value-iteration-function-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Value iteration function</a></span></li><li><span><a href="#Evaluate-result" data-toc-modified-id="Evaluate-result-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Evaluate result</a></span><ul class="toc-item"><li><span><a href="#Optimal-Policy" data-toc-modified-id="Optimal-Policy-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Optimal Policy</a></span></li><li><span><a href="#Optimal-Value-Functions" data-toc-modified-id="Optimal-Value-Functions-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Optimal Value Functions</a></span></li></ul></li><li><span><a href="#Test-the-result" data-toc-modified-id="Test-the-result-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Test the result</a></span></li></ul></div>

# Imports

In [2]:
import numpy as np 
import gym.spaces
from lib.envs.gridworld import GridworldEnv

# Create Environment

    X is your position
    T is the terminal position

In [3]:
env = GridworldEnv()

In [4]:
env._render()

T  o  o  o
o  o  o  o
o  o  o  o
o  o  o  x


# Value iteration function

In [7]:
def value_iteration(env, epsilon=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """
    def one_step_lookahead(V, a, s):

        [(prob, next_state, reward, done)] = env.P[s][a]
        v = prob * (reward + discount_factor * V[next_state])

        return v

    #start with inital value function and intial policy
    V = np.zeros(env.nS)
    policy = np.zeros([env.nS, env.nA])

    #while not the optimal policy
    while True:
      #for stopping condition
        delta = 0

        #loop over state space
        for s in range(env.nS):

            actions_values = np.zeros(env.nA)

            #loop over possible actions
            for a in range(env.nA):

                #apply bellman eqn to get actions values
                actions_values[a] = one_step_lookahead(V, a, s)

            #pick the best action
            best_action_value = max(actions_values)

            #get the biggest difference between best action value and our old value function
            delta = max(delta, abs(best_action_value - V[s]))

            #apply bellman optimality eqn
            V[s] = best_action_value

            #to update the policy
            best_action = np.argmax(actions_values)

            #update the policy
            policy[s] = np.eye(env.nA)[best_action]


        #if optimal value function
        if(delta < epsilon):
            break

    return policy, V

In [8]:
policy, v = value_iteration(env, discount_factor=1)

# Evaluate result

## Optimal Policy
    0 = up
    1 = right
    2 = down
    3 = left

In [17]:
p = np.reshape(np.argmax(policy, axis=1), env.shape)


print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(p)
print("")

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]



## Optimal Value Functions

In [18]:
print("Value Function:")
print(v)
print("")

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]



In [19]:
print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



# Test the result

In [14]:
env.reset()
env._render()

T  o  o  o
o  o  o  o
o  x  o  o
o  o  o  T


In [15]:
print('start point')
env._render()
while True:
    result = env.step(np.argmax(policy[env.s]))
    print('moving\n')
    env._render()
    if(result[2]):
        print('destination')
        break

start point
T  o  o  o
o  o  o  o
o  x  o  o
o  o  o  T
moving

T  o  o  o
o  x  o  o
o  o  o  o
o  o  o  T
moving

T  x  o  o
o  o  o  o
o  o  o  o
o  o  o  T
moving

x  o  o  o
o  o  o  o
o  o  o  o
o  o  o  T
destination
